In [1]:
import pandas as pd
import pyodbc

# Initialize connection parameters

In [2]:
DB = {'servername': 'MSI',
      'source': 'AdventureWorks2019',
      'destination':'DW',
      'user':'user1',
      'pass':'123456'}

# Create connection

In [3]:
#SourceDB
sourceConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['source'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')

#DestinationDB
destConn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + DB['servername'] + ';DATABASE='+DB['destination'] +';UID='+ DB['user']+';PWD='+DB['pass']+ ';Trusted_Connection=yes')
cursor=destConn.cursor()

In [46]:
# for index, row in date.iterrows():
#       cursor.execute('''INSERT INTO dbo.DateDim (date) values(?)''', row.ModifiedDate)
#       cursor.commit()
# datedim2 = pd.read_sql('SELECT * FROM dbo.DateDim',destConn)
# print(datedim2)


# Create DIM tables

In [5]:
cursor.execute('''CREATE TABLE DimDate (
    FullDate date  primary key,
    DayOfWeek varchar(10)
)''')
cursor.commit()


In [6]:
cursor.execute('''CREATE TABLE DimShipMethod (
    ShipMethodID int  primary key,
    Name varchar(50)
)''')

cursor.execute('''CREATE TABLE DimProduct (
    ProductID int primary key,
    Name varchar(50),
    Color varchar(50),
    Size varchar(50),
    Class varchar(50),
    Style varchar(50)
)''')

cursor.execute('''CREATE TABLE DimEmployee (
    EmployeeID int primary key,
    Name varchar(50)
)''')

cursor.execute('''CREATE TABLE DimVendor (
    VendorID int primary key,
    Name varchar(50),
    CreditRating varchar(20)
)''')

cursor.execute('''CREATE TABLE DimPurchasingStatus (
    PurchasingStatusID int primary key,
    Name varchar(50)
)''')


cursor.execute('''CREATE TABLE DimTerritory (
    TerritoryID int primary key,
    Name varchar(50),
    CountryRegionCode varchar(5),
    Group varchar(20),
)''')

cursor.execute('''CREATE TABLE DimSalesStatus (
    SalesStatusID int primary key,
    Name varchar(50)
)''')

cursor.execute('''CREATE TABLE DimOnlineOrderFlag (
    OnlineOrderFlagID int primary key,
    Name varchar(50)
)''')

cursor.commit()


# Create Facts

In [7]:
cursor.execute('''CREATE TABLE FactPurchasingOrders (
    PurchaseOrderID int  not null,
    PurchaseOrderDetailID int  not null,
    ProductID int FOREIGN KEY REFERENCES DimProduct(ProductID),
    StatusID int FOREIGN KEY REFERENCES DimPurchasingStatus(PurchasingStatusID),
    EmployeeID int FOREIGN KEY REFERENCES DimEmployee(EmployeeID),
    VendorID int FOREIGN KEY REFERENCES DimVendor(VendorID),
    ShipMethodID int FOREIGN KEY REFERENCES DimShipMethod(ShipMethodID),
    OrderDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    ShipDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    DueDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    LineTotal money,
    ReceivedQty decimal(8,2),
    RejectedQty decimal(8,2),
    StockQty decimal(9,2)
    CONSTRAINT PK_PurchasingOrder PRIMARY KEY (PurchaseOrderID,PurchaseOrderDetailID)
)''')
cursor.commit()


In [8]:
cursor.execute('''CREATE TABLE FactSalesOrders (
    SalesOrderID int  not null,
    SalesOrderDetailID int  not null,
    ProductID int FOREIGN KEY REFERENCES DimProduct(ProductID),
    StatusID int FOREIGN KEY REFERENCES DimSalesStatus(SalesStatusID),
    TerritoryID int FOREIGN KEY REFERENCES DimCustomer(TerritoryID),
    ShipMethodID int FOREIGN KEY REFERENCES DimShipMethod(ShipMethodID),
    OnlineOrderFlagID int FOREIGN KEY REFERENCES DimOnlineOrderFlag(OnlineOrderFlagID),
    OrderDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    DueDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    ShipDate date FOREIGN KEY REFERENCES DimDate(FullDate),
    LineTotal money,
    OrderQty int,
    CONSTRAINT PK_SalesOrder PRIMARY KEY (SalesOrderID,SalesOrderDetailID)

)''')
cursor.commit()
